# LangGraph Concurrent Interrupts - Graph State Analysis

This notebook helps you understand and visualize the state of your LangGraph agent with concurrent interrupts.

## Overview

Your agent implements:
- **React Agent v2** with parallel execution
- **Concurrent interrupts** using the agent-inbox pattern
- **Human-in-the-loop** capabilities with structured interrupt handling
- **Two tools**: `fast_analysis_tool` and `slow_processing_tool`

In [2]:
import sys
import os
from pathlib import Path
import json
import time
from datetime import datetime

# Add src directory to path
sys.path.append('src')

from agent import build_agent, HumanInterrupt, ActionRequest, HumanInterruptConfig
from langgraph.types import Command
from langgraph.checkpoint.memory import MemorySaver

## 1. Initialize the Agent

Create an agent instance with memory checkpointer for state persistence.

In [3]:
# Build agent with local checkpointer
agent = build_agent(local_checkpointer=True)

# Configuration for this session
config = {
    "configurable": {"thread_id": "notebook_session"},
    "recursion_limit": 50
}

print("✅ Agent initialized with memory checkpointer")
print(f"📝 Thread ID: {config['configurable']['thread_id']}")

2025-08-19 15:17:58 - agent - INFO - Created React Agent v2 with parallel execution support


✅ Agent initialized with memory checkpointer
📝 Thread ID: notebook_session


## 2. Graph State Inspection Functions

Helper functions to inspect and visualize the agent's state.

In [4]:
def display_state_info(state):
    """Display comprehensive state information."""
    print("🔍 GRAPH STATE ANALYSIS")
    print("=" * 50)
    
    # Basic state info
    print(f"📊 State Keys: {list(state.values.keys()) if state.values else 'None'}")
    print(f"🎯 Next Nodes: {state.next if state.next else 'None (completed)'}")
    print(f"⚠️  Interrupts: {len(state.interrupts)} active")
    
    # Message history
    if 'messages' in state.values and state.values['messages']:
        print(f"\n💬 MESSAGES ({len(state.values['messages'])} total):")
        for i, msg in enumerate(state.values['messages'][-3:], 1):  # Show last 3
            role = getattr(msg, 'type', 'unknown')
            content = str(getattr(msg, 'content', msg))[:100] + '...' if len(str(getattr(msg, 'content', msg))) > 100 else str(getattr(msg, 'content', msg))
            print(f"  {i}. [{role}]: {content}")
    
    # Interrupt details
    if state.interrupts:
        print(f"\n⏸️  ACTIVE INTERRUPTS:")
        for i, interrupt in enumerate(state.interrupts, 1):
            print(f"  {i}. ID: {interrupt.id}")
            print(f"     Value: {json.dumps(interrupt.value, indent=6) if interrupt.value else 'None'}")
    
    print("\n" + "=" * 50)

def display_interrupt_details(interrupts):
    """Display detailed interrupt information for human review."""
    if not interrupts:
        print("📝 No interrupts to display")
        return
    
    print("\n🚨 INTERRUPT DETAILS FOR HUMAN REVIEW")
    print("=" * 60)
    
    for i, interrupt in enumerate(interrupts, 1):
        if interrupt.value:
            try:
                # Parse interrupt value as HumanInterrupt
                interrupt_data = interrupt.value
                print(f"\n📋 Interrupt #{i}")
                print(f"   🆔 ID: {interrupt_data.get('id', 'N/A')}")
                print(f"   📝 Description: {interrupt_data.get('description', 'N/A')}")
                
                action_req = interrupt_data.get('action_request', {})
                print(f"   🔧 Action: {action_req.get('action', 'N/A')}")
                print(f"   📊 Args: {json.dumps(action_req.get('args', {}), indent=8)}")
                
                config = interrupt_data.get('config', {})
                print(f"   ⚙️  Allowed Actions:")
                print(f"      • Accept: {config.get('allow_accept', False)}")
                print(f"      • Edit: {config.get('allow_edit', False)}")
                print(f"      • Ignore: {config.get('allow_ignore', False)}")
                print(f"      • Respond: {config.get('allow_respond', False)}")
                
            except Exception as e:
                print(f"   ❌ Error parsing interrupt: {e}")
                print(f"   📄 Raw value: {interrupt.value}")
    
    print("\n" + "=" * 60)

## 3. Test Concurrent Execution with Interrupts

Run a query that triggers both tools concurrently to see how interrupts work.

In [5]:
# Test query that will trigger both tools concurrently
test_query = "Run fast_analysis_tool on 'sales-data-Q4' and slow_processing_tool on 'customer-feedback-data' with mode 'detailed' concurrently"

print(f"🚀 STARTING CONCURRENT EXECUTION")
print(f"📝 Query: {test_query}")
print("=" * 70)

start_time = datetime.now()

# Start execution
response = agent.invoke(
    {"messages": [("user", test_query)]},
    config=config,
)

print(f"\n⏱️  Initial execution time: {(datetime.now() - start_time).total_seconds():.2f}s")

# Get current state
state = agent.get_state(config)
display_state_info(state)

🚀 STARTING CONCURRENT EXECUTION
📝 Query: Run fast_analysis_tool on 'sales-data-Q4' and slow_processing_tool on 'customer-feedback-data' with mode 'detailed' concurrently


2025-08-19 15:18:02 - httpx - INFO - HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
2025-08-19 15:18:02 - agent - INFO - [FAST_TOOL] Starting analysis of: sales-data-Q4
2025-08-19 15:18:02 - agent - INFO - [SLOW_TOOL] Starting detailed processing of: customer-feedback-data
2025-08-19 15:18:02 - agent - INFO - [FAST_TOOL] Sleeping for 2 seconds...
2025-08-19 15:18:02 - agent - INFO - [SLOW_TOOL] Processing mode: detailed
2025-08-19 15:18:02 - agent - INFO - [SLOW_TOOL] Sleeping for 2 seconds...
2025-08-19 15:18:04 - agent - INFO - [FAST_TOOL] Interrupting for human approval with ID: 38b983ec-fdfb-443b-ae31-f2597f4b2a55
2025-08-19 15:18:04 - agent - INFO - [SLOW_TOOL] Interrupting for human approval with ID: e435acd4-33a1-4e29-b88b-475fb426da73



⏱️  Initial execution time: 4.85s
🔍 GRAPH STATE ANALYSIS
📊 State Keys: ['messages']
🎯 Next Nodes: ('tools', 'tools')
⚠️  Interrupts: 2 active

💬 MESSAGES (2 total):
  1. [human]: Run fast_analysis_tool on 'sales-data-Q4' and slow_processing_tool on 'customer-feedback-data' with ...
  2. [ai]: 

⏸️  ACTIVE INTERRUPTS:
  1. ID: feddc0c963b2e33571a6335b979956d1
     Value: {
      "id": "38b983ec-fdfb-443b-ae31-f2597f4b2a55",
      "action_request": {
            "action": "fast_analysis",
            "args": {
                  "data": "sales-data-Q4"
            }
      },
      "config": {
            "allow_ignore": true,
            "allow_respond": false,
            "allow_edit": true,
            "allow_accept": true
      },
      "description": "Fast analysis tool needs approval to complete analysis of: sales-data-Q4."
}
  2. ID: 258a584755b8b7e2a833d1941e8ac9a4
     Value: {
      "id": "e435acd4-33a1-4e29-b88b-475fb426da73",
      "action_request": {
            "action": "sl

## 4. Handle Interrupts

Display interrupt details and provide response options.

In [6]:
# Display interrupt details
display_interrupt_details(state.interrupts)

if state.interrupts:
    print("\n🎮 RESPONSE OPTIONS:")
    print("=" * 30)
    print('1. Accept all: {"type": "accept"}')
    print('2. Ignore all: {"type": "ignore"}')
    print('3. Edit parameters: {"type": "edit", "args": {"param": "new_value"}}')
    print('4. Custom response per interrupt')
else:
    print("\n✅ No interrupts - execution completed!")


🚨 INTERRUPT DETAILS FOR HUMAN REVIEW

📋 Interrupt #1
   🆔 ID: 38b983ec-fdfb-443b-ae31-f2597f4b2a55
   📝 Description: Fast analysis tool needs approval to complete analysis of: sales-data-Q4.
   🔧 Action: fast_analysis
   📊 Args: {
        "data": "sales-data-Q4"
}
   ⚙️  Allowed Actions:
      • Accept: True
      • Edit: True
      • Ignore: True
      • Respond: False

📋 Interrupt #2
   🆔 ID: e435acd4-33a1-4e29-b88b-475fb426da73
   📝 Description: Slow processing tool needs approval for detailed processing of: customer-feedback-data.
   🔧 Action: slow_processing
   📊 Args: {
        "data": "customer-feedback-data",
        "mode": "detailed"
}
   ⚙️  Allowed Actions:
      • Accept: True
      • Edit: True
      • Ignore: True
      • Respond: False


🎮 RESPONSE OPTIONS:
1. Accept all: {"type": "accept"}
2. Ignore all: {"type": "ignore"}
3. Edit parameters: {"type": "edit", "args": {"param": "new_value"}}
4. Custom response per interrupt


## 5. Resume Execution with Different Response Strategies

Try different response strategies to see how they affect execution.

In [7]:
# Strategy 1: Accept all interrupts
if state.interrupts:
    print("\n🟢 STRATEGY 1: ACCEPT ALL INTERRUPTS")
    print("=" * 40)
    
    resume_map = {
        interrupt.id: {"type": "accept"}
        for interrupt in state.interrupts
    }
    
    print(f"📤 Resume map: {json.dumps(resume_map, indent=2)}")
    
    # Resume execution
    resume_start = datetime.now()
    final_response = agent.invoke(
        Command(resume=resume_map),
        config=config,
    )
    
    print(f"\n⏱️  Resume execution time: {(datetime.now() - resume_start).total_seconds():.2f}s")
    print(f"🏁 Total execution time: {(datetime.now() - start_time).total_seconds():.2f}s")
    
    # Show final result
    if final_response and 'messages' in final_response:
        last_message = final_response['messages'][-1]
        print(f"\n💬 Final Response: {getattr(last_message, 'content', last_message)}")
    
    # Show updated state
    final_state = agent.get_state(config)
    display_state_info(final_state)
else:
    print("\n✅ No interrupts to handle - execution already completed!")

2025-08-19 15:18:14 - agent - INFO - [FAST_TOOL] Starting analysis of: sales-data-Q4
2025-08-19 15:18:14 - agent - INFO - [FAST_TOOL] Sleeping for 2 seconds...
2025-08-19 15:18:14 - agent - INFO - [SLOW_TOOL] Starting detailed processing of: customer-feedback-data
2025-08-19 15:18:14 - agent - INFO - [SLOW_TOOL] Processing mode: detailed
2025-08-19 15:18:14 - agent - INFO - [SLOW_TOOL] Sleeping for 2 seconds...



🟢 STRATEGY 1: ACCEPT ALL INTERRUPTS
📤 Resume map: {
  "feddc0c963b2e33571a6335b979956d1": {
    "type": "accept"
  },
  "258a584755b8b7e2a833d1941e8ac9a4": {
    "type": "accept"
  }
}


2025-08-19 15:18:16 - agent - INFO - [FAST_TOOL] Interrupting for human approval with ID: 8c44eb35-ebb1-44b5-8c69-8feaf1c81721
2025-08-19 15:18:16 - agent - INFO - [SLOW_TOOL] Interrupting for human approval with ID: ed0df3b3-8db6-421f-abed-6474cc8d4003
2025-08-19 15:18:16 - agent - INFO - [FAST_TOOL] Received response for interrupt 8c44eb35-ebb1-44b5-8c69-8feaf1c81721: {'type': 'accept'}
2025-08-19 15:18:16 - agent - INFO - [SLOW_TOOL] Received response for interrupt ed0df3b3-8db6-421f-abed-6474cc8d4003: {'type': 'accept'}
2025-08-19 15:18:16 - agent - INFO - [FAST_TOOL] Response type: accept
2025-08-19 15:18:16 - agent - INFO - [SLOW_TOOL] Response type: accept
2025-08-19 15:18:16 - agent - INFO - [FAST_TOOL] Analysis approved
2025-08-19 15:18:16 - agent - INFO - [SLOW_TOOL] Processing approved
2025-08-19 15:18:16 - agent - INFO - [FAST_TOOL] Total execution time: 2 seconds
2025-08-19 15:18:16 - agent - INFO - [SLOW_TOOL] Total execution time: 2 seconds
2025-08-19 15:18:17 - httpx - 


⏱️  Resume execution time: 3.64s
🏁 Total execution time: 18.13s

💬 Final Response: The operations were completed successfully:

1. **Fast Analysis**: Completed on `sales-data-Q4` - Decision: approved.
2. **Slow Processing**: Completed on `customer-feedback-data` with mode `detailed` - Decision: approved.
🔍 GRAPH STATE ANALYSIS
📊 State Keys: ['messages']
🎯 Next Nodes: None (completed)
⚠️  Interrupts: 0 active

💬 MESSAGES (5 total):
  1. [tool]: Fast analysis completed: sales-data-Q4 - Decision: approved
  2. [tool]: Slow processing completed: customer-feedback-data (detailed) - Decision: approved
  3. [ai]: The operations were completed successfully:

1. **Fast Analysis**: Completed on `sales-data-Q4` - De...



## 6. Alternative Response Strategy - Edit Parameters

Demonstrate how to edit tool parameters during interrupt handling.

In [8]:
# Start a new session to test parameter editing
config_edit_test = {
    "configurable": {"thread_id": "edit_test_session"},
    "recursion_limit": 50
}

print("\n🧪 TESTING PARAMETER EDITING")
print("=" * 40)

# Test with a simpler query
edit_query = "Use slow_processing_tool to process 'test-data' with default mode"

print(f"📝 Query: {edit_query}")

# Start execution
response = agent.invoke(
    {"messages": [("user", edit_query)]},
    config=config_edit_test,
)

# Get state and check for interrupts
edit_state = agent.get_state(config_edit_test)

if edit_state.interrupts:
    print("\n🎯 Found interrupts - will demonstrate parameter editing")
    display_interrupt_details(edit_state.interrupts)
    
    # Edit the mode parameter
    resume_map = {
        interrupt.id: {
            "type": "edit",
            "args": {"mode": "quick"}  # Change from 'full' to 'quick'
        }
        for interrupt in edit_state.interrupts
    }
    
    print(f"\n🔧 EDITING PARAMETERS")
    print(f"📤 Resume map with edits: {json.dumps(resume_map, indent=2)}")
    
    # Resume with edits
    edited_response = agent.invoke(
        Command(resume=resume_map),
        config=config_edit_test,
    )
    
    # Show result
    if edited_response and 'messages' in edited_response:
        last_message = edited_response['messages'][-1]
        print(f"\n💬 Result with edited parameters: {getattr(last_message, 'content', last_message)}")
else:
    print("\n❌ No interrupts found - execution completed without interruption")


🧪 TESTING PARAMETER EDITING
📝 Query: Use slow_processing_tool to process 'test-data' with default mode


2025-08-19 15:18:38 - httpx - INFO - HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
2025-08-19 15:18:38 - agent - INFO - [SLOW_TOOL] Starting full processing of: test-data
2025-08-19 15:18:38 - agent - INFO - [SLOW_TOOL] Processing mode: full
2025-08-19 15:18:38 - agent - INFO - [SLOW_TOOL] Sleeping for 2 seconds...
2025-08-19 15:18:40 - agent - INFO - [SLOW_TOOL] Interrupting for human approval with ID: b9413241-7a82-420e-aae2-df1aecaf486d
2025-08-19 15:18:40 - agent - INFO - [SLOW_TOOL] Starting full processing of: test-data
2025-08-19 15:18:40 - agent - INFO - [SLOW_TOOL] Processing mode: full
2025-08-19 15:18:40 - agent - INFO - [SLOW_TOOL] Sleeping for 2 seconds...



🎯 Found interrupts - will demonstrate parameter editing

🚨 INTERRUPT DETAILS FOR HUMAN REVIEW

📋 Interrupt #1
   🆔 ID: b9413241-7a82-420e-aae2-df1aecaf486d
   📝 Description: Slow processing tool needs approval for full processing of: test-data.
   🔧 Action: slow_processing
   📊 Args: {
        "data": "test-data",
        "mode": "full"
}
   ⚙️  Allowed Actions:
      • Accept: True
      • Edit: True
      • Ignore: True
      • Respond: False


🔧 EDITING PARAMETERS
📤 Resume map with edits: {
  "db59772bceb702beb67e4c7783d29959": {
    "type": "edit",
    "args": {
      "mode": "quick"
    }
  }
}


2025-08-19 15:18:42 - agent - INFO - [SLOW_TOOL] Interrupting for human approval with ID: 2c301bed-40f0-4f06-9c48-a43701d8d8b1
2025-08-19 15:18:42 - agent - INFO - [SLOW_TOOL] Received response for interrupt 2c301bed-40f0-4f06-9c48-a43701d8d8b1: {'type': 'edit', 'args': {'mode': 'quick'}}
2025-08-19 15:18:42 - agent - INFO - [SLOW_TOOL] Response type: edit
2025-08-19 15:18:42 - agent - INFO - [SLOW_TOOL] Processing mode changed from 'full' to 'quick'
2025-08-19 15:18:42 - agent - INFO - [SLOW_TOOL] Total execution time: 2 seconds
2025-08-19 15:18:43 - httpx - INFO - HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"



💬 Result with edited parameters: The processing of 'test-data' has been completed in quick mode. The decision to modify the processing mode was successfully executed.


## 7. State History and Persistence

Explore how state persists across multiple interactions.

In [9]:
print("\n📚 STATE HISTORY ANALYSIS")
print("=" * 40)

# Check state history for main session
try:
    state_history = agent.get_state_history(config)
    
    print(f"📊 Available state snapshots:")
    
    for i, state_snapshot in enumerate(state_history):
        if i >= 5:  # Limit to first 5 snapshots
            break
            
        print(f"\n  Snapshot {i+1}:")
        print(f"    🆔 Config: {state_snapshot.config}")
        print(f"    📅 Created: {state_snapshot.created_at}")
        print(f"    🎯 Next: {state_snapshot.next}")
        print(f"    ⚠️  Interrupts: {len(state_snapshot.interrupts)}")
        
        if state_snapshot.values and 'messages' in state_snapshot.values:
            msg_count = len(state_snapshot.values['messages'])
            print(f"    💬 Messages: {msg_count}")
            
except Exception as e:
    print(f"❌ Error getting state history: {e}")

print("\n" + "=" * 40)


📚 STATE HISTORY ANALYSIS
📊 Available state snapshots:

  Snapshot 1:
    🆔 Config: {'configurable': {'thread_id': 'notebook_session', 'checkpoint_ns': '', 'checkpoint_id': '1f07d28e-12a5-6f70-8003-aa7292fc5453'}}
    📅 Created: 2025-08-19T18:18:17.829970+00:00
    🎯 Next: ()
    ⚠️  Interrupts: 0
    💬 Messages: 5

  Snapshot 2:
    🆔 Config: {'configurable': {'thread_id': 'notebook_session', 'checkpoint_ns': '', 'checkpoint_id': '1f07d28e-0354-6530-8002-d2eef33e07b4'}}
    📅 Created: 2025-08-19T18:18:16.223650+00:00
    🎯 Next: ('agent',)
    ⚠️  Interrupts: 0
    💬 Messages: 4

  Snapshot 3:
    🆔 Config: {'configurable': {'thread_id': 'notebook_session', 'checkpoint_ns': '', 'checkpoint_id': '1f07d28d-80b5-6ad4-8001-45d703b6dff7'}}
    📅 Created: 2025-08-19T18:18:02.527195+00:00
    🎯 Next: ('tools', 'tools')
    ⚠️  Interrupts: 2
    💬 Messages: 2

  Snapshot 4:
    🆔 Config: {'configurable': {'thread_id': 'notebook_session', 'checkpoint_ns': '', 'checkpoint_id': '1f07d28d-6765-6f

## 8. Interactive State Exploration

Tools for exploring and understanding your graph state in real-time.

In [10]:
def explore_state_interactive():
    """Interactive state exploration function."""
    current_state = agent.get_state(config)
    
    print("\n🔍 INTERACTIVE STATE EXPLORER")
    print("=" * 50)
    
    # Quick stats
    stats = {
        "Total Messages": len(current_state.values.get('messages', [])) if current_state.values else 0,
        "Active Interrupts": len(current_state.interrupts),
        "Next Nodes": len(current_state.next) if current_state.next else 0,
        "State Keys": len(current_state.values.keys()) if current_state.values else 0,
        "Thread ID": config['configurable']['thread_id']
    }
    
    for key, value in stats.items():
        print(f"📊 {key}: {value}")
    
    return current_state

# Run interactive exploration
current_state = explore_state_interactive()


🔍 INTERACTIVE STATE EXPLORER
📊 Total Messages: 5
📊 Active Interrupts: 0
📊 Next Nodes: 0
📊 State Keys: 1
📊 Thread ID: notebook_session


## 9. Custom Interrupt Handling Scenarios

Test different interrupt response combinations.

In [11]:
# Test mixed responses (accept some, ignore others)
def test_mixed_responses():
    """Test mixed interrupt responses."""
    config_mixed = {
        "configurable": {"thread_id": "mixed_response_test"},
        "recursion_limit": 50
    }
    
    print("\n🎭 MIXED RESPONSE TESTING")
    print("=" * 40)
    
    # Query that will create multiple interrupts
    mixed_query = "Analyze 'dataset-X' with fast tool and process 'dataset-Y' with slow tool using 'full' mode"
    
    response = agent.invoke(
        {"messages": [("user", mixed_query)]},
        config=config_mixed,
    )
    
    state = agent.get_state(config_mixed)
    
    if len(state.interrupts) >= 2:
        print(f"✅ Created {len(state.interrupts)} interrupts for mixed testing")
        
        # Mixed response: accept first, ignore second (if exists)
        resume_map = {}
        for i, interrupt in enumerate(state.interrupts):
            if i == 0:
                resume_map[interrupt.id] = {"type": "accept"}
                print(f"📝 Interrupt {i+1}: ACCEPT")
            else:
                resume_map[interrupt.id] = {"type": "ignore"}
                print(f"📝 Interrupt {i+1}: IGNORE")
        
        # Resume with mixed responses
        final_response = agent.invoke(
            Command(resume=resume_map),
            config=config_mixed,
        )
        
        # Show results
        if final_response and 'messages' in final_response:
            last_message = final_response['messages'][-1]
            print(f"\n🎯 Mixed response result: {getattr(last_message, 'content', last_message)}")
    else:
        print(f"⚠️  Only {len(state.interrupts)} interrupts created - mixed testing needs 2+")

# Run mixed response test
test_mixed_responses()


🎭 MIXED RESPONSE TESTING


2025-08-19 15:19:13 - httpx - INFO - HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
2025-08-19 15:19:13 - agent - INFO - [SLOW_TOOL] Starting full processing of: dataset-Y
2025-08-19 15:19:13 - agent - INFO - [SLOW_TOOL] Processing mode: full
2025-08-19 15:19:13 - agent - INFO - [FAST_TOOL] Starting analysis of: dataset-X
2025-08-19 15:19:13 - agent - INFO - [SLOW_TOOL] Sleeping for 2 seconds...
2025-08-19 15:19:13 - agent - INFO - [FAST_TOOL] Sleeping for 2 seconds...
2025-08-19 15:19:15 - agent - INFO - [FAST_TOOL] Interrupting for human approval with ID: 9291177e-2964-48bd-b2ba-b5c524192f19
2025-08-19 15:19:15 - agent - INFO - [SLOW_TOOL] Interrupting for human approval with ID: 4f55dcdb-986e-4833-b494-0eecca35537f
2025-08-19 15:19:15 - agent - INFO - [FAST_TOOL] Starting analysis of: dataset-X
2025-08-19 15:19:15 - agent - INFO - [SLOW_TOOL] Starting full processing of: dataset-Y
2025-08-19 15:19:15 - agent - INFO - [FAST_TOOL] Sleeping for 2 second

✅ Created 2 interrupts for mixed testing
📝 Interrupt 1: ACCEPT
📝 Interrupt 2: IGNORE


2025-08-19 15:19:17 - agent - INFO - [FAST_TOOL] Interrupting for human approval with ID: 1fd379b5-8e1e-480c-a5ac-9b5cb228e69e
2025-08-19 15:19:17 - agent - INFO - [SLOW_TOOL] Interrupting for human approval with ID: 8153ac23-aead-46fb-a5fe-ea04a86e1776
2025-08-19 15:19:17 - agent - INFO - [FAST_TOOL] Received response for interrupt 1fd379b5-8e1e-480c-a5ac-9b5cb228e69e: {'type': 'accept'}
2025-08-19 15:19:17 - agent - INFO - [SLOW_TOOL] Received response for interrupt 8153ac23-aead-46fb-a5fe-ea04a86e1776: {'type': 'ignore'}
2025-08-19 15:19:17 - agent - INFO - [FAST_TOOL] Response type: accept
2025-08-19 15:19:17 - agent - INFO - [SLOW_TOOL] Response type: ignore
2025-08-19 15:19:17 - agent - INFO - [FAST_TOOL] Analysis approved
2025-08-19 15:19:17 - agent - INFO - [FAST_TOOL] Total execution time: 2 seconds
2025-08-19 15:19:17 - agent - WARNING - [SLOW_TOOL] Processing cancelled by user
2025-08-19 15:19:17 - agent - INFO - [SLOW_TOOL] Total execution time: 2 seconds
2025-08-19 15:19:1


🎯 Mixed response result: The results of the operations are as follows:

- **Fast Analysis:** Completed for `dataset-X` - Decision: approved.
- **Slow Processing:** Cancelled for `dataset-Y`.


## 10. Summary and Key Insights

Summary of your LangGraph concurrent interrupts implementation.

In [12]:
print("\n📋 LANGGRAPH CONCURRENT INTERRUPTS SUMMARY")
print("=" * 60)

summary_points = [
    "🤖 Agent Type: React Agent v2 with parallel execution",
    "🛠️  Tools: fast_analysis_tool, slow_processing_tool",
    "⏸️  Interrupt Pattern: Agent-inbox with structured requests",
    "🔄 State Management: MemorySaver checkpointer",
    "🎯 Concurrent Execution: Multiple tools can interrupt simultaneously",
    "📝 Response Types: accept, ignore, edit, respond",
    "🆔 Interrupt Tracking: UUID-based interrupt identification",
    "📊 State Persistence: Full conversation and interrupt history"
]

for point in summary_points:
    print(f"  {point}")

print("\n🔑 KEY FEATURES:")
print("  • Each tool can interrupt independently")
print("  • Human can accept, ignore, or edit tool parameters")
print("  • State persists across interrupt/resume cycles")
print("  • Concurrent tool execution with individual approval")
print("  • Structured interrupt data with configuration options")

print("\n🎮 USAGE PATTERNS:")
print("  1. Send query → Tools execute → Interrupts triggered")
print("  2. Review interrupt details → Choose response strategy")
print("  3. Resume with Command(resume=response_map)")
print("  4. Continue until completion or next interrupt")

print("\n" + "=" * 60)
print("✅ Notebook analysis complete!")
print("📝 Use this notebook to explore different interrupt scenarios")


📋 LANGGRAPH CONCURRENT INTERRUPTS SUMMARY
  🤖 Agent Type: React Agent v2 with parallel execution
  🛠️  Tools: fast_analysis_tool, slow_processing_tool
  ⏸️  Interrupt Pattern: Agent-inbox with structured requests
  🔄 State Management: MemorySaver checkpointer
  🎯 Concurrent Execution: Multiple tools can interrupt simultaneously
  📝 Response Types: accept, ignore, edit, respond
  🆔 Interrupt Tracking: UUID-based interrupt identification
  📊 State Persistence: Full conversation and interrupt history

🔑 KEY FEATURES:
  • Each tool can interrupt independently
  • Human can accept, ignore, or edit tool parameters
  • State persists across interrupt/resume cycles
  • Concurrent tool execution with individual approval
  • Structured interrupt data with configuration options

🎮 USAGE PATTERNS:
  1. Send query → Tools execute → Interrupts triggered
  2. Review interrupt details → Choose response strategy
  3. Resume with Command(resume=response_map)
  4. Continue until completion or next inter